In [1]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key :
    raise ValueError("请设置环境变量 DEEPSEEK_API_KEY")


text_lines = []
with open("./mfd.md", 'r', encoding='utf-8') as file:
   for line in file:
       if line is not None and line.strip() != "":
           text_lines.append(line)
           
print(len(text_lines))
print(text_lines[10])
# 初始化deepseek 接口客户端对象
from openai import OpenAI
deepseek_client = OpenAI(
    api_key = api_key,
    base_url="https://api.deepseek.com/v1"
)

# 准备向量数据库
# from pymilvus import model as milvus_model
# embedding_model = milvus_model.DefaultEmbeddingFunction()

697
**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。



In [2]:
from sentence_transformers import SentenceTransformer

# 专用中文嵌入模型
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def chinese_embedding(texts):
    return model.encode(texts).tolist()
    

In [3]:
import numpy as np

# 插入前归一化
def normalize_vectors(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

question = "不动产登记机构不得有哪些行为"
query_vector = normalize_vectors([chinese_embedding([question])])[0]
len(query_vector[0])

384

In [5]:
from pymilvus import MilvusClient
from pymilvus import utility
from tqdm import tqdm
milvus_client = MilvusClient(uri="./milvus_civil_code_2.db")
collection_name = "civil_code_collection"

# 执行删除
field_name = "vector"  # 向量字段名
# safe_delete_index(collection_name, field_name)

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(query_vector[0]),
    metric_type="IP",
    consistency_level="Strong"
)

# index_params = {
#     "index_type": "HNSW",  # 对高维中文更友好
#     "metric_type": "IP", 
#     "params": {
#         "M": 24,     # 更高连通性 (16-48)
#         "efConstruction": 200  # 更精细构建
#     }
# }

# milvus_client.create_index(
#     collection_name=collection_name,
#     field_name="vector",
#     index_params=index_params
# )

data = []
doc_embeddings = normalize_vectors(chinese_embedding(text_lines))
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id":i, "vector":doc_embeddings[i], "text": line})
# print(data[0:10])
milvus_client.insert(collection_name=collection_name, data = data)

Creating embeddings: 100%|██████████| 697/697 [00:00<00:00, 1007037.51it/s]


{'insert_count': 697, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [ ]:
# len(embedding_model.encode_queries([question])[0])

In [6]:

search_res = milvus_client.search(
    collection_name = collection_name,
    data=query_vector,
    limit=3,
    search_param={"metric_type":"IP", "params":{}},
    output_fields=["text"]
)

print(search_res)

import json
retrieved_lines_with_distances=[
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

data: [[{'id': 21, 'distance': 12.124847412109375, 'entity': {'text': '申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。\n'}}, {'id': 31, 'distance': 11.665121078491211, 'entity': {'text': '不动产登记簿记载的事项与不动产权属证书记载的事项不一致的，除有证据证明不动产登记簿确有错误外，以不动产登记簿为准。\n'}}, {'id': 32, 'distance': 10.981340408325195, 'entity': {'text': '**第二百一十七条** 不动产权属证书是权利人享有该不动产物权的证明。不动产权属证书记载的事项，应当与不动产登记簿一致；不一致的，除有证据证明不动产登记簿确有错误外，以不动产登记簿为准。\n'}}]]
[
    [
        "\u7533\u8bf7\u767b\u8bb0\u7684\u4e0d\u52a8\u4ea7\u5b58\u5728\u5c1a\u672a\u89e3\u51b3\u7684\u6743\u5c5e\u4e89\u8bae\u7684\uff0c\u767b\u8bb0\u673a\u6784\u5e94\u5f53\u4e0d\u4e88\u767b\u8bb0\uff0c\u5e76\u4e66\u9762\u544a\u77e5\u7533\u8bf7\u4eba\u3002\n",
        12.124847412109375
    ],
    [
        "\u4e0d\u52a8\u4ea7\u767b\u8bb0\u7c3f\u8bb0\u8f7d\u7684\u4e8b\u9879\u4e0e\u4e0d\u52a8\u4ea7\u6743\u5c5e\u8bc1\u4e66\u8bb0\u8f7d\u7684\u4e8b\u9879\u4e0d\u4e00\u81f4\u7684\uff0c\u9664\u6709\u8bc1\u636e\u8bc1\u660e\u4e0d\u52a8\u4ea7\u767b\u8bb0\u7c3f\u786e\u6709\u9519\u8bef\u5

In [7]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)



根据提供的上下文，不动产登记机构不得有以下行为：

1. 对存在权属争议的不动产予以登记（必须不予登记并书面告知申请人）
2. 在登记簿与权属证书记载不一致时，无证据证明登记簿错误的情况下擅自以权属证书为准（必须坚持以不动产登记簿为准）

法律依据：
- 对于权属争议的情况未明确说明具体法条，但明确规定"应当不予登记"
- 对于登记簿与权属证书记载不一致的情况，依据《民法典》第二百一十七条规定

<translated>
根据提供的上下文，不动产登记机构不得有以下行为：

1. 对存在权属争议的不动产进行登记（必须拒绝登记并书面通知申请人）
2. 当登记记录与产权证书内容不一致时，在没有证据证明登记记录确实有误的情况下，擅自以产权证书为准（必须坚持以不动产登记记录为准）

法律依据：
- 对于产权争议的情况虽未指明具体条款，但明确规定"应当不予登记"
- 对于登记记录与产权证书记载不一致的情形，依据《民法典》第217条规定
</translated>
